In [ ]:
import os
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# CONFIG 
REVIEW_PATH = r"C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\data\Appliances.jsonl"
META_PATH   = r"C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\data\meta_Appliances.jsonl"
OUT_DIR     = r"C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\output"
CHUNK_SIZE  = 200_000   # tune based on your machine's RAM
SEED        = 42

os.makedirs(OUT_DIR, exist_ok=True)

#LOAD METADATA 
meta_pd = pd.read_json(
    r"C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\data\meta_Appliances.jsonl",
    lines=True
)[["parent_asin", "average_rating", "rating_number"]]

writers = {"train": None, "valid": None, "test": None}

# STREAM, FILTER, SPLIT, AND WRITE
for chunk in pd.read_json(
    r"C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\data\Appliances.jsonl",
    lines=True,
    chunksize=CHUNK_SIZE
):
    # 1) keep needed cols + timestamp
    chunk = chunk[["user_id", "parent_asin", "rating", "timestamp", "text"]]

    # 2) filter to years 2021–2023
    dt = pd.to_datetime(chunk["timestamp"], unit="ms")
    mask_year = dt.dt.year.between(2021, 2023)
    chunk = chunk.loc[mask_year]
    if chunk.empty:
        continue

    # 3) assign random float for splitting
    rng = np.random.RandomState(SEED)
    chunk["_rand"] = rng.rand(len(chunk))

    # 4) merge metadata
    chunk = chunk.merge(meta_pd, on="parent_asin", how="left")

    # 5) define split masks
    masks = {
        "train": chunk["_rand"] <  0.8,
        "valid": (chunk["_rand"] >= 0.8) & (chunk["_rand"] < 0.9),
        "test":  chunk["_rand"] >= 0.9
    }

    # 6) write each split to its Parquet
    for split, m in masks.items():
        sub = chunk.loc[m, [
            "user_id",
            "parent_asin",
            "rating",
            "text",
            "average_rating",
            "rating_number"
        ]]
        if sub.empty:
            continue
        tbl = pa.Table.from_pandas(sub, preserve_index=False)
        path = os.path.join(OUT_DIR, f"{split}.parquet")
        if writers[split] is None:
            writers[split] = pq.ParquetWriter(path, schema=tbl.schema)
        writers[split].write_table(tbl)

# close Parquet writers
for w in writers.values():
    if w:
        w.close()

print("Finished writing filtered splits to:", OUT_DIR)
print("  •", os.path.join(OUT_DIR, "train.parquet"))
print("  •", os.path.join(OUT_DIR, "valid.parquet"))
print("  •", os.path.join(OUT_DIR, "test.parquet"))


Finished writing filtered splits to: C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\output
  • C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\output\train.parquet
  • C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\output\valid.parquet
  • C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\output\test.parquet


In [2]:
import pyarrow.parquet as pq

for split in ("train", "valid", "test"):
    path = rf"C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\output\{split}.parquet"
    pf = pq.ParquetFile(path)
    print(f"{split.capitalize()} split: {pf.metadata.num_rows} reviews")


Train split: 616463 reviews
Valid split: 77195 reviews
Test split: 76002 reviews


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CheckRatingRange") \
    .config("spark.driver.memory","4g") \
    .getOrCreate()

# Load just the rating column from the train split
train = spark.read.parquet(
    r"C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\output\train.parquet"
).select("rating")

# 1) min & max
train.selectExpr("min(rating) AS min_rating", "max(rating) AS max_rating") \
     .show()

# 2) basic summary (mean, stddev, etc.)
train.describe("rating").show()

# 3) breakdown by rating value
train.groupBy("rating").count().orderBy("rating").show(5, truncate=False)

spark.stop()


In [12]:
import os
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from pyspark.sql import SparkSession
import pyarrow as pa
import pyarrow.parquet as pq

TRAIN_PARQUET = r"C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\output\train.parquet"
EMB_OUT       = r"C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\output\embeddings\item_embeddings.parquet"
SAMPLE_PER    = 5        
BATCH_SIZE    = 64      
DEVICE        = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

os.makedirs(os.path.dirname(EMB_OUT), exist_ok=True)

spark = SparkSession.builder \
    .appName("ItemBERTEmbeddings") \
    .config("spark.driver.memory", "16g") \
    .getOrCreate()

df = spark.read.parquet(TRAIN_PARQUET).select("parent_asin", "text")
pdf = df.toPandas()   
spark.stop()

model = SentenceTransformer("all-MiniLM-L6-v2", device=DEVICE)

writer = None
schema = None

for pid, group in pdf.groupby("parent_asin", sort=False):
    texts = group["text"].sample(
        n=min(len(group), SAMPLE_PER),
        random_state=42
    ).tolist()

    embs = model.encode(
        texts,
        batch_size=BATCH_SIZE,
        show_progress_bar=False,
        convert_to_numpy=True
    )
    mean_emb = embs.mean(axis=0)

    data = {"parent_asin": [pid]}
    for i, v in enumerate(mean_emb):
        data[f"emb_{i}"] = [float(v)]
    table = pa.Table.from_pydict(data)

    if writer is None:
        schema = table.schema
        writer = pq.ParquetWriter(EMB_OUT, schema=schema)


    writer.write_table(table)


if writer:
    writer.close()

print("Wrote item embeddings to:", EMB_OUT)


Using device: cpu
Wrote item embeddings to: C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\output\embeddings\item_embeddings.parquet


In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# ─── CONFIG ──────────────────────────────────────────────────────────────────
EMB_PATH   = r"C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\output\embeddings\item_embeddings.parquet"
META_PATH  = r"C:\CUNY_MSDS\DATA612\AMAZON_PROJECT\data\meta_Appliances.jsonl"
TOP_K      = 5

# ─── 1) Load embeddings and metadata ─────────────────────────────────────────
df_emb = pd.read_parquet(EMB_PATH)
df_meta = pd.read_json(META_PATH, lines=True)[["parent_asin", "title"]]
df_meta = df_meta.rename(columns={"parent_asin": "item_id", "title": "product_title"})

# ─── 2) Merge to get titles alongside embeddings ─────────────────────────────
df = df_emb.rename(columns={"parent_asin": "item_id"}).merge(df_meta, on="item_id", how="left")

# ─── 3) Fit Nearest Neighbors on embedding vectors ────────────────────────────
X = df.filter(regex="^emb_").values
item_ids = df["item_id"].values
titles   = df["product_title"].values

nn = NearestNeighbors(n_neighbors=TOP_K+1, metric="cosine")
nn.fit(X)

# ─── 4) Recommendation function returning titles ─────────────────────────────
def recommend_titles(item_id: str, top_k: int = TOP_K):
    if item_id not in item_ids:
        raise ValueError(f"Item ID {item_id} not found.")
    idx = np.where(item_ids == item_id)[0][0]
    distances, indices = nn.kneighbors([X[idx]], n_neighbors=top_k+1)
    rec_idxs = indices[0][1:]
    return titles[rec_idxs].tolist()

# ─── 5) Show for a sample item ───────────────────────────────────────────────
sample_id = item_ids[:5]
print("Sample item:")
print(" - ID:   ", sample_id)
print(" - Title:", titles[0])
print("\nTop 5 similar items by title:")
for rank, pt in enumerate(recommend_titles(sample_id), start=1):
    print(f"{rank}. {pt}")


Sample item:
 - ID:    B07DD37QPZ
 - Title: Essential Values 18 Pack Compatible Replacement Filters (90 Dryer Loads Total) for Bettervent Indoor Dryer Vent

Top 5 similar items by title:
1. Polyester Lint Trap Dryer Filter Replacement Part Compatible With Bettervent Indoor Electric Dryers Vent Kit Pack of 12
2. 97006931 Range Hood Aluminum Filter for Compatible with Kenmore Broan S97006931 1172137 5-3078 88150 99010121 C88150 K758900 Range Hood Filter BP29 (2 pack)
3. VBENLEM Pack of 6 Hood Filters 19.5W x 15.5H Inch, 430 Stainless Steel 5 Grooves Commercial Hood Filters, Range Hood Filter for Grease Rated Commercial Kitchen Exhaust Hoods
4. Refrigerator Air Filter Replacement for LG LT120F Kenmore Elite 46-9918, 469918, 9918, Replace ADQ73214402, ADQ73214403, ADQ73214404, ADQ73334008-6 Pack
5. Colorfullife 6 Pack Humidifier Wicking Filter T for Honeywell Top Fill Tower Humidifier HEV615, HEV620, Replacement Filter T, Replace Parts HFT600T HFT600PDQ
